In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from tqdm.notebook import tqdm
from perlin_noise import PerlinNoise
import os

In [ ]:
size = (100, 200)

# initialize heigh map with perlin noise
height_map0 = np.zeros(size)
x = np.arange(size[0])
y = np.arange(size[1])
Y, X = np.meshgrid(y, x)

noise = PerlinNoise(octaves=3, seed=1)
y_ridge_mean = 20
ridge_wobble = 10
wobble_noise = PerlinNoise(octaves=3, seed=2)
y_ridge = y_ridge_mean + ridge_wobble*np.array([wobble_noise(i/size[0]) for i in range(size[0])])
y_sigma = 10
ridge_height = 10

for i in tqdm(range(size[0])):
    for j in range(size[1]):
        height_map0[i,j] = ridge_height/(1 + (j - y_ridge[i])**2 / y_sigma**2) + noise([i/size[0], j/size[1]]) + 1 + 0.05*(size[1] - y_ridge_mean - np.abs(j - y_ridge_mean))

# circular hole
# height_map0 = -np.exp(-np.sqrt((X - 0.5*size[0])**2 + (Y-0.5*size[1])**2)/20)

# parabolic valley
# height_map0 = np.sqrt((X - 0.5*size[0])**2 + 1) + Y


plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

plt.imshow(height_map0, cmap='gray')
plt.colorbar()

fig.tight_layout()

In [ ]:
# compute the gradient of the height map
grad = np.gradient(-height_map0)
# height_map0_grad = np.sqrt(height_map0_grad[0]**2 + height_map0_grad[1]**2)


In [ ]:
plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

# plt.imshow(height_map0, cmap='gray')
plt.pcolormesh(X,Y,height_map0, cmap='gray')
plt.axis('equal')
plt.colorbar()

plt.quiver(X,Y,grad[0],grad[1], color='red')

fig.tight_layout()

## Erosion dynamics

At each grid-cell $k=(i,j)$ we want to track several quantities:

- The height of the cell $H$
- The water content $Q \geq 0$
- The suspended sediment content $S < \alpha Q$
- The flow velocity $V^x, V^y$

Now we want to write differential equations for all quantities:

$$ \dot{H}_k(t) = deposition - erosion $$

$$ \dot{Q}_k(t) = \Delta_k(t) + flow $$

$$ \dot{S}_k(t) = erosion - deposition + flow $$

$$ \dot{V}_k(t) = height gradient - friction + exported momentum $$

Let us start by simply letting water flow without altering the height map.
Let us also suppose for now that there is no inertia, so that the velocity field is simply the gradient

In [ ]:
Q = np.ones_like(height_map0)
eta = 10


H = np.copy(height_map0)
# Q = np.zeros_like(height_map0)
V = np.stack(np.gradient(-H), axis=-1)

Qs = [np.copy(Q)]
# iterate
for i in tqdm(range(100)):
    # V = np.stack(np.gradient(-H), axis=-1)
    V_mod = np.abs(V[...,0]) + np.abs(V[...,1])
    repartition = np.abs(V[...,0])/V_mod # fraction of flow in x direction at each point
    repartition[V_mod == 0] = 0.5

    export = Q*np.minimum(V_mod*eta, 1)

    left_exports = np.roll((V[...,0] < 0)*export*repartition, -1, axis=0)
    left_exports[-1,:] = 0
    right_exports = np.roll((V[...,0] > 0)*export*repartition, 1, axis=0)
    right_exports[0,:] = 0
    down_exports = np.roll((V[...,1] < 0)*export*(1 - repartition), -1, axis=1)
    down_exports[:,-1] = 0
    up_exports = np.roll((V[...,1] > 0)*export*(1 - repartition), 1, axis=1)
    up_exports[:,0] = 0

    dQ = -export + left_exports + right_exports + down_exports + up_exports

    Q += dQ

    Qs.append(np.copy(Q))

    if np.abs(dQ).max() < 1e-5:
        break




In [ ]:
np.max(Qs[-1])

In [ ]:
folder = 'erosion-movie-1'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(Qs))):

    if i % 1 == 0:
        plt.close(1)
        fig, ax = plt.subplots(num=1, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        plt.pcolormesh(X,Y,Qs[i], cmap='Blues', vmin=0, vmax=1.5)
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

In [ ]:
plt.close('all')

## Accounting for water height

The previous simulation has water accumulating in the lowest pixels. But that is unrealistic, as we want rather the formation of lakes, where the water fills the lowest basins in the height map.
However, we need to be careful when we modify the height map to avoid the formation of waves.

### Test anti-slosh

In [ ]:
# test water pour
height_map0 = np.zeros_like(height_map0)
Q = np.zeros_like(height_map0)
Q[Q.shape[0]//2, Q.shape[1]//2] = 100
Q[Q.shape[0]//2+1, Q.shape[1]//2] = 100
Q[Q.shape[0]//2, Q.shape[1]//2+1] = 100
Q[Q.shape[0]//2+1, Q.shape[1]//2+1] = 100

In [ ]:
Q = np.ones_like(height_map0)
water_height = 0.1
eta = 10

anti_slosh = 3


H = np.copy(height_map0) + water_height*Q
# Q = np.zeros_like(height_map0)
V = np.stack(np.gradient(-H), axis=-1)

def roll(array, direction='right'):
    if direction == 'left':
        a = np.roll(array, -1, axis=0)
        a[-1,:] = 0
    elif direction == 'right':
        a = np.roll(array, 1, axis=0)
        a[0,:] = 0
    elif direction == 'down':
        a = np.roll(array, -1, axis=1)
        a[:,-1] = 0
    elif direction == 'up':
        a = np.roll(array, 1, axis=1)
        a[:,0] = 0
    else:
        raise ValueError()

    return a

Qs = [np.copy(Q)]
# iterate
for i in tqdm(range(3000)):
    V = np.stack(np.gradient(-H), axis=-1)
    V_mod = np.abs(V[...,0]) + np.abs(V[...,1])
    repartition = np.abs(V[...,0])/V_mod # fraction of flow in x direction at each point
    repartition[V_mod == 0] = 0.5

    max_export = Q*np.minimum(V_mod*eta, 1)

    max_left_export = np.maximum((H - roll(H, 'right'))/water_height/(anti_slosh + 1), 0)
    max_right_export = np.maximum((H - roll(H, 'left'))/water_height/(anti_slosh + 1), 0)
    max_down_export = np.maximum((H - roll(H, 'up'))/water_height/(anti_slosh + 1), 0)
    max_up_export = np.maximum((H - roll(H, 'down'))/water_height/(anti_slosh + 1), 0)

    left_exports = np.minimum((V[...,0] < 0)*max_export*repartition, max_left_export)
    right_exports = np.minimum((V[...,0] > 0)*max_export*repartition, max_right_export)
    down_exports = np.minimum((V[...,1] < 0)*max_export*(1 - repartition), max_down_export)
    up_exports = np.minimum((V[...,1] > 0)*max_export*(1 - repartition), max_up_export)

    export = left_exports + right_exports + down_exports + up_exports

    left_exports = roll(left_exports, 'left')
    right_exports = roll(right_exports, 'right')
    down_exports = roll(down_exports, 'down')
    up_exports = roll(up_exports, 'up')

    dQ = -export + left_exports + right_exports + down_exports + up_exports

    Q += dQ
    H += water_height*dQ

    Qs.append(np.copy(Q))

    

In [ ]:
folder = 'erosion-movie-4'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(Qs))):

    if i % 1 == 0:
        plt.close(2)
        fig, ax = plt.subplots(num=2, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        plt.pcolormesh(X,Y,Qs[i], cmap='Blues', vmin=0, vmax=1.5)
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

In [ ]:
folder = 'erosion-movie-height-4'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(Qs))):

    if i % 30 == 0:
        plt.close(2)
        fig, ax = plt.subplots(num=2, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        plt.pcolormesh(X,Y,height_map0 + water_height*Qs[i], cmap='gray',
                        # vmin=0, vmax=1.5
                        )
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

### Trying a different approach

Anti slosh can still generate waves because multiple neighbors can come to fill the same grid point, allowing it to become higher than its neighbors and initiating oscillations.
Also, using the gradient will get us stuck in symmetric situations.

We can simplify computing the height differences between neighboring pixels and then the flow. Also, it is quite easy to add precipitation and evaporation

In [ ]:
# test water pour
height_map0 = np.zeros_like(height_map0)
Q = np.zeros_like(height_map0)
Q[Q.shape[0]//2, Q.shape[1]//2] = 100
Q[Q.shape[0]//2+1, Q.shape[1]//2] = 100
Q[Q.shape[0]//2, Q.shape[1]//2+1] = 100
Q[Q.shape[0]//2+1, Q.shape[1]//2+1] = 100

In [ ]:
Q = np.ones_like(height_map0)
water_height = 0.1
eta = 0.1
eps_tol = 1e-5

P_freq = 0
evaporation = 0.0

H = np.copy(height_map0) + water_height*Q
# H = H + water_height*Q

directions = ['left', 'down', 'right', 'up']
def roll(array, direction=0):
    if direction in [0, 'left']:
        a = np.roll(array, -1, axis=0)
        a[-1,:] = 0
    elif direction in [1, 'down']:
        a = np.roll(array, -1, axis=1)
        a[:,-1] = 0
    elif direction in [2, 'right']:
        a = np.roll(array, 1, axis=0)
        a[0,:] = 0
    elif direction in [3, 'up']:
        a = np.roll(array, 1, axis=1)
        a[:,0] = 0
    else:
        raise ValueError()

    return a

Qs = [np.copy(Q)]
# iterate
for i in tqdm(range(30000)):

    # the exports to the left are computed comparing to the left neighbor, which requires rolling the height map right
    exports = eta*np.stack([np.maximum((H - roll(H, ((d+2) % 4)))/water_height, 0) for d in range(4)], axis=-1) 

    total_export = np.sum(exports, axis=-1)
    # adjust export to be less than Q
    total_export_ = np.minimum(total_export, Q)
    total_export[total_export == 0] = 1
    exports = (exports.T * total_export_.T / total_export.T).T
    total_export = total_export_

    # compute imports by properly rolling exports
    imports = np.stack([roll(exports[...,d], d) for d in range(4)], axis=-1)

    total_import = np.sum(imports, axis=-1)

    # evaporation
    E = 0
    if evaporation:
        E = np.ones_like(Q) * (Q > 0) * evaporation

    # precipitation
    P = 0
    if P_freq:
        P = np.zeros_like(Q)
        if i % P_freq == 0:
            P += 1


    dQ = total_import - total_export + P - E

    # clip so that Q stays above 0
    dQ = np.maximum(Q + dQ, 0) - Q

    if np.max(np.abs(dQ)) < eps_tol:
        break

    Q += dQ
    H += water_height*dQ

    Qs.append(np.copy(Q))

    

In [ ]:
np.sum(Qs[100])

In [ ]:
folder = 'erosion-movie-14'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(Qs))):

    if i % 100 == 0:
        plt.close(2)
        fig, ax = plt.subplots(num=2, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        plt.pcolormesh(X,Y,Qs[i], cmap='Blues', vmin=0, vmax=1.5)
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 

plt.close(1)
fig = plt.figure(num=1, figsize=(9,6))
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(X, Y, height_map0 - 0.01, color='gray')
ax.plot_surface(X, Y, height_map0 + water_height*Qs[-1], color='blue', alpha=0.5)


fig.tight_layout()

In [ ]:
plt.close(1)
fig, ax = plt.subplots(num=1, figsize=(9,6))

plt.plot([np.sum(Q_) for Q_ in Qs], label='Q')

fig.tight_layout()

This seems to work pretty well, though we might have problems with inertia. But maybe it is good to ditch the idea of inertia... we'll see

## Erosion

A simple way to do erosion is by eroding proportionally to the water that flows through each cell. We can compute this easily as `total_export + total_import`

In [ ]:
Q = np.ones_like(height_map0)
water_height = 0.01
eta = 0.1
eps_tol = 1e-5

P_freq = 87
evaporation = 0.02
erosion = 0.0001


H = np.copy(height_map0)

directions = ['left', 'down', 'right', 'up']
def roll(array, direction=0):
    if direction in [0, 'left']:
        a = np.roll(array, -1, axis=0)
        a[-1,:] = 0
    elif direction in [1, 'down']:
        a = np.roll(array, -1, axis=1)
        a[:,-1] = 0
    elif direction in [2, 'right']:
        a = np.roll(array, 1, axis=0)
        a[0,:] = 0
    elif direction in [3, 'up']:
        a = np.roll(array, 1, axis=1)
        a[:,0] = 0
    else:
        raise ValueError()

    return a

Qs = [np.copy(Q)]
Hs = [np.copy(H)]
# iterate
for i in tqdm(range(30000)):

    H_ = H + water_height*Q # height accounting water level

    # the exports to the left are computed comparing to the left neighbor, which requires rolling the height map right
    exports = eta*np.stack([np.maximum((H_ - roll(H_, ((d+2) % 4)))/water_height, 0) for d in range(4)], axis=-1)
    bedrock_diffs = eta*np.stack([np.maximum((H - roll(H, ((d+2) % 4))), 0) for d in range(4)], axis=-1)

    max_erosion = np.max(bedrock_diffs, axis=-1) # we cannot erode more than the lowest neighbor

    total_export = np.sum(exports, axis=-1)
    # adjust export to be less than Q
    total_export_ = np.minimum(total_export, Q)
    total_export[total_export == 0] = 1
    exports = (exports.T * total_export_.T / total_export.T).T
    total_export = total_export_

    # compute imports by properly rolling exports
    imports = np.stack([roll(exports[...,d], d) for d in range(4)], axis=-1)

    total_import = np.sum(imports, axis=-1)

    # evaporation
    E = 0
    if evaporation:
        E = np.ones_like(Q) * (Q > 0) * evaporation

    # precipitation
    P = 0
    if P_freq:
        P = np.zeros_like(Q)
        if i % P_freq == 0:
            P += 1


    dQ = total_import - total_export + P - E

    flow = total_import + total_export

    # clip so that Q stays above 0
    dQ = np.maximum(Q + dQ, 0) - Q

    if np.max(np.abs(dQ)) < eps_tol:
        break

    Q += dQ
    H += -np.minimum(erosion*flow, max_erosion)

    Qs.append(np.copy(Q))
    Hs.append(np.copy(H))


In [ ]:
from mpl_toolkits.mplot3d import Axes3D 

plt.close(1)
fig = plt.figure(num=1, figsize=(9,6))
ax = fig.add_subplot(111, projection='3d')

# ax.plot_surface(X, Y, height_map0, color='gray')
ax.plot_surface(X, Y, Hs[-1], color='red')


fig.tight_layout()

In [ ]:
folder = 'erosion-movie-15'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(Qs))):

    if i % 300 == 0:
        plt.close(2)
        fig, ax = plt.subplots(num=2, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        plt.pcolormesh(X,Y,Hs[i], cmap='gray')
        # plt.pcolormesh(X,Y,Qs[i], cmap='Blues', vmin=0, vmax=1.5)
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

This is very promising. We can try to make the flow 8 directional to have smoother edges...

In [ ]:
Q = np.ones_like(height_map0)
water_height = 0.01
eta = 0.1
eps_tol = 1e-5

P_freq = 87
evaporation = 0.02
erosion = 0.0001


H = np.copy(height_map0)

directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
lengths = np.array([1,np.sqrt(2)]*4)
def roll(array, direction=0, fill_value=0):
    if direction in [0, 'N']:
        a = np.roll(array, 1, axis=1)
        a[:,0] = fill_value
    elif direction in [2, 'E']:
        a = np.roll(array, 1, axis=0)
        a[0,:] = fill_value
    elif direction in [4, 'S']:
        a = np.roll(array, -1, axis=1)
        a[:,-1] = fill_value
    elif direction in [6, 'W']:
        a = np.roll(array, -1, axis=0)
        a[-1,:] = fill_value
    elif direction in [1, 'NE']:
        return roll(roll(array, direction='N', fill_value=fill_value), direction='E', fill_value=fill_value)
    elif direction in [3, 'SE']:
        return roll(roll(array, direction='S', fill_value=fill_value), direction='E', fill_value=fill_value)
    elif direction in [5, 'SW']:
        return roll(roll(array, direction='S', fill_value=fill_value), direction='W', fill_value=fill_value)
    elif direction in [7, 'NW']:
        return roll(roll(array, direction='N', fill_value=fill_value), direction='W', fill_value=fill_value)
    else:
        raise ValueError()

    return a

Qs = [np.copy(Q)]
Hs = [np.copy(H)]
# iterate
for i in tqdm(range(30000)):

    H_ = H + water_height*Q # height accounting water level

    # the exports to the left are computed comparing to the left neighbor, which requires rolling the height map right
    exports = eta*np.stack([np.maximum((H_ - roll(H_, ((d+4) % 8)))/water_height, 0) for d in range(8)], axis=-1)/lengths
    bedrock_diffs = eta*np.stack([np.maximum((H - roll(H, ((d+4) % 8))), 0) for d in range(8)], axis=-1)

    max_erosion = np.max(bedrock_diffs, axis=-1) # we cannot erode more than the lowest neighbor

    total_export = np.sum(exports, axis=-1)
    # adjust export to be less than Q
    total_export_ = np.minimum(total_export, Q)
    total_export[total_export == 0] = 1
    exports = (exports.T * total_export_.T / total_export.T).T
    total_export = total_export_

    # compute imports by properly rolling exports
    imports = np.stack([roll(exports[...,d], d) for d in range(8)], axis=-1)

    total_import = np.sum(imports, axis=-1)

    # evaporation
    E = 0
    if evaporation:
        E = np.ones_like(Q) * (Q > 0) * evaporation

    # precipitation
    P = 0
    if P_freq:
        P = np.zeros_like(Q)
        if i % P_freq == 0:
            P += 1


    dQ = total_import - total_export + P - E

    flow = total_import + total_export

    # clip so that Q stays above 0
    dQ = np.maximum(Q + dQ, 0) - Q

    if np.max(np.abs(dQ)) < eps_tol:
        break

    Q += dQ
    H += -np.minimum(erosion*flow, max_erosion)

    Qs.append(np.copy(Q))
    Hs.append(np.copy(H))


In [ ]:
folder = 'erosion-movie-17'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(Qs))):

    if i % 300 == 0:
        plt.close(2)
        fig, ax = plt.subplots(num=2, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        plt.pcolormesh(X,Y,Hs[i], cmap='gray')
        # plt.pcolormesh(X,Y,Qs[i], cmap='Blues', vmin=0, vmax=1.5)
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 

plt.close(1)
fig = plt.figure(num=1, figsize=(9,6))
ax = fig.add_subplot(111, projection='3d')

# ax.plot_surface(X, Y, height_map0, color='gray')
ax.plot_surface(X, Y, Hs[-1], color='red')


fig.tight_layout()

What if we force the flow to go only to one neighbor?

In [ ]:
Q = np.ones_like(height_map0)
water_height = 0.01
eta = 0.1
eps_tol = 1e-5

P_freq = 87
evaporation = 0.02
erosion = 0.0001


H = np.copy(height_map0)

directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
lengths = np.array([1,np.sqrt(2)]*4)
def roll(array, direction=0, fill_value=0):
    if direction in [0, 'N']:
        a = np.roll(array, 1, axis=1)
        a[:,0] = fill_value
    elif direction in [2, 'E']:
        a = np.roll(array, 1, axis=0)
        a[0,:] = fill_value
    elif direction in [4, 'S']:
        a = np.roll(array, -1, axis=1)
        a[:,-1] = fill_value
    elif direction in [6, 'W']:
        a = np.roll(array, -1, axis=0)
        a[-1,:] = fill_value
    elif direction in [1, 'NE']:
        return roll(roll(array, direction='N', fill_value=fill_value), direction='E', fill_value=fill_value)
    elif direction in [3, 'SE']:
        return roll(roll(array, direction='S', fill_value=fill_value), direction='E', fill_value=fill_value)
    elif direction in [5, 'SW']:
        return roll(roll(array, direction='S', fill_value=fill_value), direction='W', fill_value=fill_value)
    elif direction in [7, 'NW']:
        return roll(roll(array, direction='N', fill_value=fill_value), direction='W', fill_value=fill_value)
    else:
        raise ValueError()

    return a

Qs = [np.copy(Q)]
Hs = [np.copy(H)]
# iterate
for i in tqdm(range(30000)):

    H_ = H + water_height*Q # height accounting water level

    # the exports to the left are computed comparing to the left neighbor, which requires rolling the height map right
    exports = eta*np.stack([np.maximum((H_ - roll(H_, ((d+4) % 8)))/water_height, 0) for d in range(8)], axis=-1)/lengths
    bedrock_diffs = eta*np.stack([np.maximum((H - roll(H, ((d+4) % 8))), 0) for d in range(8)], axis=-1)

    max_erosion = np.max(bedrock_diffs, axis=-1) # we cannot erode more than the lowest neighbor

    # set to zero all exports but the maximum
    max_vals = np.max(exports, axis=-1, keepdims=True)
    exports[exports < max_vals] = 0

    total_export = np.sum(exports, axis=-1)


    # adjust export to be less than Q
    total_export_ = np.minimum(total_export, Q)
    total_export[total_export == 0] = 1
    exports = (exports.T * total_export_.T / total_export.T).T
    total_export = total_export_

    # compute imports by properly rolling exports
    imports = np.stack([roll(exports[...,d], d) for d in range(8)], axis=-1)

    total_import = np.sum(imports, axis=-1)

    # evaporation
    E = 0
    if evaporation:
        E = np.ones_like(Q) * (Q > 0) * evaporation

    # precipitation
    P = 0
    if P_freq:
        P = np.zeros_like(Q)
        if i % P_freq == 0:
            P += 1


    dQ = total_import - total_export + P - E

    flow = total_import + total_export

    # clip so that Q stays above 0
    dQ = np.maximum(Q + dQ, 0) - Q

    if np.max(np.abs(dQ)) < eps_tol:
        break

    Q += dQ
    H += -np.minimum(erosion*flow, max_erosion)

    Qs.append(np.copy(Q))
    Hs.append(np.copy(H))

In [ ]:
folder = 'erosion-movie-18'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(Qs))):

    if i % 300 == 0:
        plt.close(2)
        fig, ax = plt.subplots(num=2, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        plt.pcolormesh(X,Y,Hs[i], cmap='gray')
        # plt.pcolormesh(X,Y,Qs[i], cmap='Blues', vmin=0, vmax=1.5)
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

Nej, I don't like it very much

## Adding sediment transport

`H` now will be a 3 dimensional array where we stack the thicknesses of bedrock, deposited sediment, loose sediment and water

Also, to make things more orderly, we import Roll from erosion module

In [ ]:
import sys
sys.path.append('../fractalcoast/')
from erosion import *

In [ ]:
class Precipitation:
    def __init__(self, intensity, freq, pattern):
        self.intensity = intensity
        self.freq = freq
        self.pattern = pattern

    def __call__(self, t):
        if self.intensity == 0 or self.freq == 0:
            return 0
        if t % self.freq == 0:
            return self.intensity * self.pattern
        else:
            return 0

class Eroder:
    def __init__(self, height_map0, precipitation_model=None, neighbors=8):
        if len(height_map0.shape) != 2:
            raise ValueError()
        self.layers = np.zeros(height_map0.shape + (4,))
        self.layers[...,0] = height_map0
        self.history = []
        self.t = 0

        self.precipitation_model=precipitation_model

        if neighbors == 4:
            self.roll = Roll4() # rolling function
        elif neighbors == 8:
            self.roll = Roll8()
        else:
            raise ValueError()
        
        self.params = [
            'eta', # flow speed scale factor
            'solubility', # max sediment content (i.e. water hardness)
            'erosion', # erosion rate
            'bedrock_hardness', # how much harder it is to erode bedrock with respect to sediment
            'deposition', # how much sediment in still water to deposit (1 means all)
            'evaporation', # evaporation rate
            'eps_tol', # stop running if the max change in Q is less than eps_tol
            ]
        
        self.param_history = []
        
        for k in self.params:
            self.__dict__[k] = None

        self._height = None
        self._rock_height = None
        self._fluid_height = None

        self.do_break = False

    def reset_cache(self):
        self._height = None
        self._rock_height = None
        self._fluid_height = None

    def set_precipitation_model(self, precipitation_model):
        self.precipitation_model = precipitation_model

    @property
    def bedrock(self):
        return self.layers[...,0]
    
    @property
    def sandstone(self):
        return self.layers[...,1]
    
    @property
    def S(self): # suspended sediment
        return self.layers[...,2]
    
    @property
    def Q(self): # water
        return self.layers[...,-1]
    
    @property
    def height(self):
        if self._height is None:
            self._height = np.sum(self.layers, axis=-1)
        return self._height
    
    @property
    def rock_height(self):
        if self._rock_height is None:
            self._rock_height = self.bedrock + self.sandstone
        return self._rock_height
    
    @property
    def fluid_height(self):
        if self._fluid_height is None:
            self._fluid_height = self.S + self.Q
        return self._fluid_height
    
    @property
    def water_hardness(self): # fraction of sediment in fluid
        fluid_height = np.copy(self.fluid_height)
        fluid_height[fluid_height == 0] = 1
        return self.S/fluid_height
    

    def step(self, record_history=False):

        # compute tentative fluid export
        exports = self.eta*np.stack([np.maximum((self.height - self.roll(self.height, self.roll.reverse_direction(d))), 0) for d in range(len(self.roll))], axis=-1)/self.roll.lengths

        # compute differences in rock height to constrain erosion and deposition
        rock_diffs = np.stack([self.rock_height - self.roll(self.rock_height, self.roll.reverse_direction(d)) for d in range(len(self.roll))], axis=-1)
        
        max_erosion = np.max(rock_diffs, axis=-1) # we cannot erode more than the lowest neighbor
        max_erosion[max_erosion < 0] = 0 # negative erosion doesn't make sense

        total_export = np.sum(exports, axis=-1)
        # adjust export to be less than fluid height
        total_export_ = np.minimum(total_export, self.fluid_height)
        total_export[total_export == 0] = 1
        exports = (exports.T * total_export_.T / total_export.T).T
        total_export = total_export_

        S_exports = (exports.T * self.water_hardness.T).T
        Q_exports = exports - S_exports

        total_S_export = np.sum(S_exports, axis=-1)
        total_Q_export = np.sum(Q_exports, axis=-1)

        # compute imports by properly rolling exports
        S_imports = np.stack([self.roll(S_exports[...,d], d) for d in range(len(self.roll))], axis=-1)
        Q_imports = np.stack([self.roll(Q_exports[...,d], d) for d in range(len(self.roll))], axis=-1)

        total_S_import = np.sum(S_imports, axis=-1)
        total_Q_import = np.sum(Q_imports, axis=-1)

        total_import = total_S_import + total_Q_import


        ## erosion
        erosion_potential = (total_import + total_export)*self.erosion

        # print(f'{erosion_potential.shape = }, {self.sandstone.shape = }, {max_erosion.shape = }')

        # start by eroding sediment
        eroded_sediment = np.minimum(np.minimum(erosion_potential, self.sandstone), max_erosion)
        erosion_potential -= eroded_sediment
        max_erosion -= eroded_sediment
        # erode bedrock (this happens only if we eroded all the sandstone and we have still erosion potential left)
        eroded_bedrock = np.minimum(erosion_potential/self.bedrock_hardness, max_erosion)

        E = 0
        if self.evaporation:
            E = np.ones_like(self.Q) * (self.Q > 0) * self.evaporation

        P = self.precipitation_model(self.t)

        # update layer thicknesses
        self.bedrock[...] -= eroded_bedrock
        self.sandstone[...] -= eroded_sediment
        self.S[...] += total_S_import - total_S_export + eroded_sediment + eroded_bedrock
        dQ = total_Q_import - total_Q_export + P - E
        self.Q[...] += dQ

        # clip negative values of Q
        self.Q[self.Q < 0] = 0

        ## deposition

        # immediately deposit sediment in oversaturated water
        max_S = self.Q * self.solubility
        deposition1 = np.maximum(self.S - max_S, 0)
        self.S[...] -= deposition1

        # compute fraction of still water and deposit from it
        still_water = np.maximum(self.Q - total_Q_export - total_Q_import, 0)
        Q = np.copy(self.Q)
        Q[Q == 0] = 1
        deposition2 = self.deposition*self.S*still_water/Q
        self.S[...] -= deposition2
        
        self.sandstone[...] += deposition1 + deposition2

        # record history
        if record_history:
            self.history.append((self.t, np.copy(self.layers)))
        
        # update
        self.reset_cache()
        self.t +=1

        if np.max(np.abs(dQ)) < self.eps_tol:
            self.do_break = True

    def run(self, iterations=100, record_every=10, eta=None, solubility=None, erosion=None, bedrock_hardness=None,
            deposition=None, evaporation=None, eps_tol=None):
        return self._run(iterations, record_every, eta=eta, solubility=solubility, erosion=erosion, bedrock_hardness=bedrock_hardness,
                         deposition=deposition,  evaporation=evaporation, eps_tol=eps_tol)

    def _run(self, iterations=100, record_every=10, **kwargs):
        
        param_history_item = {}
        for k,v in kwargs.items():
            if k in self.params:
                if v is not None and v != self.__dict__[k]:
                    self.__dict__[k] = v
                    param_history_item[k] = v
            else:
                print(f'Ignoring kwarg {k}')

        for k in self.params + ['precipitation_model']:
            if self.__dict__[k] is None:
                raise ValueError(f'Value of {k} is not set: cannot run')
            
        if param_history_item:    
            self.param_history.append((self.t, param_history_item))

        for i in tqdm(range(iterations)):
            self.step(record_history=(record_every and not (i % record_every)))

            if self.do_break:
                break

        self.do_break = False



In [ ]:
prec = Precipitation(0.01, 89, np.ones_like(height_map0))
eroder = Eroder(height_map0, precipitation_model=prec)

eroder.run(
    iterations=30000,
    record_every=100,
    eta=0.1,
    solubility=1,
    erosion=0.01,
    bedrock_hardness=10,
    deposition=0.1,
    evaporation=2e-4,
    eps_tol=1e-4
)

In [ ]:
folder = 'erosion-movie-20'
if not os.path.exists(folder):
    os.makedirs(folder)
else:
    raise FileExistsError()

frame = 1
for i in tqdm(range(len(eroder.history))):

    if i % 3 == 0:
        plt.close(2)
        fig, ax = plt.subplots(num=2, figsize=(9,6))

        # plt.imshow(height_map0, cmap='gray')
        # plt.pcolormesh(X,Y,np.sum(eroder.history[i][1][...,:2], axis=-1), cmap='gray')
        plt.pcolormesh(X,Y,eroder.history[i][1][...,0] - eroder.history[0][1][...,0], cmap='gray')
        # plt.pcolormesh(X,Y,Qs[i], cmap='Blues', vmin=0, vmax=1.5)
        plt.axis('equal')
        plt.colorbar()

        fig.tight_layout()

        fig.savefig(f'{folder}/{frame:04d}.png', dpi=200)
        frame += 1

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 

plt.close(1)
fig = plt.figure(num=1, figsize=(9,6))
ax = fig.add_subplot(111, projection='3d')

# ax.plot_surface(X, Y, height_map0, color='gray')
ax.plot_surface(X, Y, eroder.history[-1][1][...,0],
                # color=eroder.history[-1][1][...,0] - eroder.history[0][1][...,0],
                cmap='gray')


fig.tight_layout()